In [6]:
% matplotlib inline

import numpy as np
import scipy as sp
import math
import scipy.stats as scp
import matplotlib as mpl
from numpy import linalg as la
from numpy import random as rand
from matplotlib import pyplot as plt
from sklearn import svm

In [3]:
Data = np.genfromtxt("TrainingRidge.csv", delimiter=',', skip_header=True,dtype='float').T

X = Data[0:2,:].T
Y = Data[2,:][np.newaxis,:].T


In [41]:
# train
nsvm = svm.NuSVC(kernel='rbf',nu=0.1)
nsvm.fit(X,Y)

#predict 
pred_y = c_svm.predict(X)
pred_y.astype(int)
print pred_y

colormap = np.array(['r', 'g', 'b'])
plt.scatter(X[:,0],X[:,1],c=colormap[pred_y.astype(int)])


[ 10.  14.   2.  32.  10.   9.  15.  19.  10.  33.  14.  35.   6.  18.  37.
  14.  12.   3.  17.  12.   7.  14.  20.  22.  29.   6.   9.  14.  41.  12.
  26.   9.  18.   6.  21.  10.   6.   8.  12.  29.  11.  14.   9.  17.  40.
  30.  26.  29.  31.  14.   9.  20.  14.  12.   6.  16.  27.  14.  10.  14.
  15.   8.  14.  20.  11.  13.  27.  16.  12.   3.   9.  11.  14.  53.  33.
  11.   7.  15.  34.  14.  13.  22.  14.  10.  23.  23.   9.  25.   9.  27.
  20.  11.  15.  12.  31.   7.  13.  12.  17.  33.  17.  19.  30.  22.  10.
  10.  15.  41.  21.   6.   5.  11.  19.  25.  13.  26.  33.   9.  24.  15.
  14.  10.  16.   9.  16.  24.  32.  17.  18.  17.  15.  20.  11.  15.  11.
   9.  12.  13.  38.  15.  29.  16.  22.  22.  18.   8.  27.  26.   4.  32.
  12.   6.  16.  10.  14.  12.  42.  13.  22.  12.  11.  44.   7.  17.   9.
  19.  23.   3.  12.  22.   3.  22.  13.  21.  14.   5.  29.   9.   7.  46.
  11.  32.  30.  10.   5.   7.  20.  28.  13.  14.  41.  15.  15.   6.  25.
  36.  16.  

IndexError: index 10 is out of bounds for axis 1 with size 3